In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import reduce
import os
import sys
sys.path.insert(0, '../../src/')
import data_loader
pd.set_option('display.max_columns', None)

In [ ]:
# seasons_data = data_loader.load_data()
# season_2018 = data_loader.load_data(seasons=["2018-19"])
# game_data = data_loader.load_game_data(seasons=["2018-19"])
# seasons_game_data = data_loader.load_game_data()

In [ ]:
seasons_data = data_loader.load_data()
seasons_data

In [8]:
GAME_ID_STR = "GAME_ID"
pbp_grouped = seasons_data.groupby(GAME_ID_STR)
game_data = pbp_grouped.size().to_frame(name="play_count")

game_data["HOME TEAM ID"] = pbp_grouped.apply(lambda x: x[x["PERSON1TYPE"] == 'HOME_PLAYER']["PLAYER1_TEAM_ID"].iloc[0])
game_data["VISITOR TEAM ID"] = pbp_grouped.apply(lambda x: x[x["PERSON1TYPE"] == 'VISITOR_PLAYER']["PLAYER1_TEAM_ID"].iloc[0])
game_data[["HOME_PLAYER_1", "HOME_PLAYER_2", "HOME_PLAYER_3", "HOME_PLAYER_4", "HOME_PLAYER_5", "VISITOR_PLAYER_1", "VISITOR_PLAYER_2", "VISITOR_PLAYER_3", "VISITOR_PLAYER_4", "VISITOR_PLAYER_5"]] = np.nan

errors = []

In [ ]:
for year in range(0, 19):

    file_str = ""

    if year < 9:
        file_str = "200{}-0{}".format(year, year + 1)
    elif year == 9:
        file_str = "200{}-{}".format(year, year + 1)
    else:
        file_str = "20{}-{}".format(year, year + 1)

    print("Starting loading season {}".format(file_str))
    seasons_data = data_loader.load_data(seasons=[file_str])
    print("Finished loading season {}".format(file_str))

    pbp_grouped = seasons_data.groupby("GAME_ID")
    game_data = pbp_grouped.size().to_frame(name="play_count")

    game_data["HOME TEAM ID"] = pbp_grouped.apply(lambda x: x[x["PERSON1TYPE"] == 'HOME_PLAYER']["PLAYER1_TEAM_ID"].iloc[0])
    game_data["VISITOR TEAM ID"] = pbp_grouped.apply(lambda x: x[x["PERSON1TYPE"] == 'VISITOR_PLAYER']["PLAYER1_TEAM_ID"].iloc[0])
    game_data[["HOME_PLAYER_1", "HOME_PLAYER_2", "HOME_PLAYER_3", "HOME_PLAYER_4", "HOME_PLAYER_5", "VISITOR_PLAYER_1", "VISITOR_PLAYER_2", "VISITOR_PLAYER_3", "VISITOR_PLAYER_4", "VISITOR_PLAYER_5"]] = np.nan


    for i, row in game_data.iterrows():
        errors = []

        home_team_id = row["HOME TEAM ID"]
        visitor_team_id = row["VISITOR TEAM ID"]
        
        home_players = seasons_data[(seasons_data["EVENTMSGTYPE"] == "SUBSTITUTION") & (seasons_data["GAME_ID"] == i) & (seasons_data["PERIOD"] == 1) &(seasons_data["PLAYER1_TEAM_ID"] == home_team_id)]["PLAYER1_ID"].dropna().unique()
        visitor_players = seasons_data[(seasons_data["EVENTMSGTYPE"] == "SUBSTITUTION") & (seasons_data["GAME_ID"] == i) & (seasons_data["PERIOD"] == 1) &(seasons_data["PLAYER1_TEAM_ID"] == visitor_team_id)]["PLAYER1_ID"].dropna().unique()

        # print(i)
        
        if len(home_players) < 5:
            players_1 = seasons_data[(seasons_data["GAME_ID"] == i) & (seasons_data["PERIOD"] == 1) & (seasons_data["PLAYER1_TEAM_ID"] == home_team_id)]["PLAYER1_ID"].dropna().unique()
            players_2 = seasons_data[(seasons_data["GAME_ID"] == i) & (seasons_data["PERIOD"] == 1) & (seasons_data["PLAYER2_TEAM_ID"] == home_team_id)]["PLAYER2_ID"].dropna().unique()
            players_3 = seasons_data[(seasons_data["GAME_ID"] == i) & (seasons_data["PERIOD"] == 1) & (seasons_data["PLAYER3_TEAM_ID"] == home_team_id)]["PLAYER3_ID"].dropna().unique()
            out_players = seasons_data[(seasons_data["EVENTMSGTYPE"] == "SUBSTITUTION") & (seasons_data["GAME_ID"] == i) & (seasons_data["PERIOD"] == 1) &(seasons_data["PLAYER1_TEAM_ID"] == home_team_id)]["PLAYER1_ID"].dropna().unique()
            in_players = seasons_data[(seasons_data["EVENTMSGTYPE"] == "SUBSTITUTION") & (seasons_data["GAME_ID"] == i) & (seasons_data["PERIOD"] == 1) &(seasons_data["PLAYER2_TEAM_ID"] == home_team_id)]["PLAYER2_ID"].dropna().unique()

            lineup = reduce(np.union1d, (players_1, players_2, players_3, home_players))
            lineup = np.setdiff1d(lineup, in_players)
            lineup = np.setdiff1d(lineup, out_players)
            home_players = np.union1d(lineup, out_players)
        
        if len(visitor_players) < 5:
            players_1 = seasons_data[(seasons_data["GAME_ID"] == i) & (seasons_data["PERIOD"] == 1) & (seasons_data["PLAYER1_TEAM_ID"] == visitor_team_id)]["PLAYER1_ID"].dropna().unique()
            players_2 = seasons_data[(seasons_data["GAME_ID"] == i) & (seasons_data["PERIOD"] == 1) & (seasons_data["PLAYER2_TEAM_ID"] == visitor_team_id)]["PLAYER2_ID"].dropna().unique()
            players_3 = seasons_data[(seasons_data["GAME_ID"] == i) & (seasons_data["PERIOD"] == 1) & (seasons_data["PLAYER3_TEAM_ID"] == visitor_team_id)]["PLAYER3_ID"].dropna().unique()
            out_players = seasons_data[(seasons_data["EVENTMSGTYPE"] == "SUBSTITUTION") & (seasons_data["GAME_ID"] == i) & (seasons_data["PERIOD"] == 1) &(seasons_data["PLAYER1_TEAM_ID"] == visitor_team_id)]["PLAYER1_ID"].dropna().unique()
            in_players = seasons_data[(seasons_data["EVENTMSGTYPE"] == "SUBSTITUTION") & (seasons_data["GAME_ID"] == i) & (seasons_data["PERIOD"] == 1) &(seasons_data["PLAYER2_TEAM_ID"] == visitor_team_id)]["PLAYER2_ID"].dropna().unique()

            lineup = reduce(np.union1d, (players_1, players_2, players_3, visitor_players))
            lineup = np.setdiff1d(lineup, in_players)
            lineup = np.setdiff1d(lineup, out_players)
            visitor_players = np.union1d(lineup, out_players)
        try:
            game_data.at[i, "HOME_PLAYER_1":"HOME_PLAYER_5"] = np.asarray(home_players[:5])
            game_data.at[i, "VISITOR_PLAYER_1" :] = np.asarray(visitor_players[:5])
        except:
            game_data.at[i, "HOME_PLAYER_1":"HOME_PLAYER_5"] = np.nan
            game_data.at[i, "VISITOR_PLAYER_1" :] = np.nan
            errors.append(i)

    file_name = "lineup-{}-id.csv".format(file_str)
    game_data.to_csv(file_name, index=True)

    if len(errors) > 0:
        print("CHECK ERRORS season {}".format(file_str))
        print(errors)
    else:
        print("DONE season {}".format(file_str))

In [41]:
frames = []

for year in range(0, 19):

    file_str = ""

    if year < 9:
        file_str = "lineup-200{}-0{}-id.csv".format(year, year + 1)
    elif year == 9:
        file_str = "lineup-200{}-{}-id.csv".format(year, year + 1)
    else:
        file_str = "lineup-20{}-{}-id.csv".format(year, year + 1)

    df = pd.read_csv(file_str)
    df.set_index("GAME_ID", inplace=True)
    frames.append(df)

lineup_all = pd.concat(frames)

In [45]:
lineup_all.to_csv("lineups-all-seasons.csv", index=True)